In [13]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import json
import datetime

plt.rcParams["figure.figsize"] = (20, 5)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [14]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/fit/" + timestamp
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
version_dir = "version/" + timestamp 

os.makedirs(version_dir)
timestamp

'20200815-170315'

In [15]:
dataset = pd.read_csv("data/NU_train_set.csv")
dataset.head()

,t,t+1,delta,tokenized_data
0,104289271808,104282980352,-6291456,-6291456
1,104282980352,104282984448,4096,4096
2,104282984448,104282988544,4096,4096
3,104282988544,104282992640,4096,4096
4,104282992640,104282996736,4096,4096


In [16]:
param_list = dict()

param_list["PAST_HISTORY"] = 16
param_list["FUTURE_TARGET"] = 8
param_list["BATCH_SIZE"] = 1024
param_list["EPOCHS"] = 250
param_list["BUFFER_SIZE"] = 200000

with open("version/{}/params.json".format(timestamp), "w") as p:
    json.dump(param_list, p, indent=4)

In [17]:
def generate_timeseries(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, n_feature)
        data.append(np.reshape(dataset[indices], (history_size, 5)))
        labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [18]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(dtype=np.float32)   #dtype=np.int64
encoded_data = encoder.fit_transform(dataset["tokenized_data"].values.reshape(-1, 1))
encoded_data[0], encoder.categories_

(<1x5 sparse matrix of type '<class 'numpy.float32'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 [array([-6291456,       -1,        0,     4096,  2097152], dtype=int64)])

import joblib

joblib.dump(encoder, "data/encoder.pkl")

In [19]:
x_train, y_train = generate_timeseries(encoded_data.toarray(), 0, None, param_list["PAST_HISTORY"], param_list["FUTURE_TARGET"])

#train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#train_data = train_data.cache().shuffle(param_list["BUFFER_SIZE"]).batch(param_list["BATCH_SIZE"])

In [20]:
x_train.shape

(47165, 16, 5)

In [21]:
y_train.shape

(47165, 8, 5)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(5))  #x_train.shape[-2:] , input_shape=[16, 5] , return_sequences=True
#model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.RepeatVector(8))
model.add(keras.layers.TimeDistributed(tf.keras.layers.Dense(5, activation="softmax")))
#model.add(tf.keras.layers.Flatten())
#model.add(tf.keras.layers.Dense(5, activation="softmax"))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(5)))  #x_train.shape[-2:] , input_shape=[16, 5] , return_sequences=True
model.add(tf.keras.layers.RepeatVector(8))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(5, return_sequences=True)))
model.add(keras.layers.TimeDistributed(tf.keras.layers.Dense(5, activation="softmax")))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
model_history = model.fit(x_train, y_train, batch_size=param_list["BATCH_SIZE"], validation_split=0.2, epochs=param_list["EPOCHS"])
model.save("version/{}/model.h5".format(timestamp))

l_loss: 0.0197 - val_accuracy: 0.9966
Epoch 113/250
37/37 [==============================] - 1s 25ms/step - loss: 0.0365 - accuracy: 0.9909 - val_loss: 0.0197 - val_accuracy: 0.9966
Epoch 114/250
37/37 [==============================] - 1s 21ms/step - loss: 0.0365 - accuracy: 0.9910 - val_loss: 0.0197 - val_accuracy: 0.9968
Epoch 115/250
37/37 [==============================] - 1s 21ms/step - loss: 0.0366 - accuracy: 0.9911 - val_loss: 0.0196 - val_accuracy: 0.9967
Epoch 116/250
37/37 [==============================] - 1s 22ms/step - loss: 0.0365 - accuracy: 0.9910 - val_loss: 0.0195 - val_accuracy: 0.9967
Epoch 117/250
37/37 [==============================] - 1s 22ms/step - loss: 0.0365 - accuracy: 0.9910 - val_loss: 0.0196 - val_accuracy: 0.9967
Epoch 118/250
37/37 [==============================] - 1s 37ms/step - loss: 0.0365 - accuracy: 0.9910 - val_loss: 0.0197 - val_accuracy: 0.9966
Epoch 119/250
37/37 [==============================] - 1s 22ms/step - loss: 0.0364 - accuracy: 0.9